<a href="https://colab.research.google.com/github/leandroaguazaco/data_science_portfolio/blob/main/Projects/04-Churn_Telco_Analysis/04_Churn_Telco_Analysis_03_modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1 align="center"> 4 - CHURN TELCO ANALYSIS </h1>

<div align="center">

  <img alt="Static Badge" src="https://img.shields.io/badge/active_project-true-blue">

</div>  

<h2 align="center"> 4.3 - Modeling </h2>

<div align="center">

  <img alt="Static Badge" src="https://img.shields.io/badge/active_section-true-blue">

  <img alt="Static Badge" src="https://img.shields.io/badge/section_status-in progress-green">

</div>  

<object
data="https://img.shields.io/badge/contact-Felipe_Leandro_Aguazaco-blue?style=flat&link=https%3A%2F%2Fwww.linkedin.com%2Fin%2Ffelipe-leandro-aguazaco%2F">
</object>

## a. Project summary

The aim of this project is to analyze and predict customer churn in the telco industry. The information pertains to client behavior, including in-call, out-call, and internet service consumption. There is a variable called 'Churn' that determines whether a customer churned within two weeks after canceling services. The information summarizes eight weeks of data for each telco line or client.

<h3 align="center"> <font color='orange'>NOTE: The project is distributed across multiple sections, separated into notebook files, in the following way:</font> </h3>



4.1 - Preprocessig data: load, join and clean data, and Exploratory data analysis, EDA.

4.2 - Premodeling: predict customer churn based on PyCaret library.

> <font color='gray'>  4.3 - Modeling: predict customer churn based on sklearn pipelines. </font> ✍ ▶ <font color='orange'> Current section </font>

4.4 - Analyzing and explaining predictions.

4.5 - Detecting vulneabilities in final machine learnig model.

4.6 - Model deployment with Streamlit.

## b. Install libraries

In [27]:
%%capture
!pip install polars
!pip install pyjanitor # Clean DataFrame
!pip install colorama
!pip install adjustText
!pip install scikit-optimize # BayesSearchCV
!pip install umap-learn # Dimension reduction
!pip install mlflow
!pip install pyngrok
!pip install xgboost
!pip install lightgbm
!pip install catboost

## c. Import libraries

In [28]:
%%capture
# c.1 Python Utilies
import pandas as pd
import polars as pl
import numpy as np
import math
import warnings
from scipy.stats import spearmanr
from statsmodels.stats.outliers_influence import variance_inflation_factor
import scipy.stats as stats
from janitor import clean_names, remove_empty
from colorama import Fore, Style
import rpy2
import shutil
import os
from google.colab import drive
import matplotlib.pyplot as plt
import seaborn as sns

In [178]:
%%capture
# c.2 Machine learning with sklearn

import mlflow
from pyngrok import ngrok
import xgboost as xgb
import lightgbm as lgb
import catboost as catb

  ## Sklearn
    ### Imputation
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
    ### Preporcessing
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, OrdinalEncoder, RobustScaler, PowerTransformer
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import VarianceThreshold
from sklearn.decomposition import PCA
    ### Models
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, ExtraTreesClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import BayesianRidge, LogisticRegression
from sklearn.feature_selection import SelectKBest, f_classif, mutual_info_classif # Univariate feature selection
from umap import UMAP # Dimensionallity reduction and feature selection
    ### Pipeline cache
from shutil import rmtree
from tempfile import mkdtemp
from joblib import Memory
    ### Metrics for classification
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, RocCurveDisplay
from sklearn.metrics import accuracy_score, precision_score, balanced_accuracy_score, recall_score, f1_score, jaccard_score ## Higher better
from sklearn.metrics import roc_auc_score, matthews_corrcoef, cohen_kappa_score
    ### Cross validation and Hyperparameter tuning
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
# from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.model_selection import KFold, RepeatedKFold, StratifiedKFold , RepeatedStratifiedKFold, ShuffleSplit, StratifiedShuffleSplit
from sklearn.model_selection import cross_validate, cross_val_score, cross_val_predict

In [ ]:
# c.3 Setups
%matplotlib inline
plt.style.use("ggplot")
warnings.simplefilter("ignore")

## d. Custom functions

### d.1 - Type conversions

In [31]:
# d.1 dtypes conversion and memory reduce function.
def dtype_conversion(df: pd.DataFrame = None, verbose: bool = True)-> pd.DataFrame:
    """
    Summary:
      Function to dtypes conversion and save reduce memory usage; takes a DataFrame as argument, returns DataFrame.
      For more details, visit: https://towardsdatascience.com/how-to-work-with-million-row-datasets-like-a-pro-76fb5c381cdd.
      The modifications include type casting for numerical and object variables.
    Parameters:
      df (pandas.DataFrame): DataFrame containing information.
      verbose (bool, default = True): If true, display results (conversions and warnings)
    Returns:
      pandas.DataFrame: original DataFrame with dtypes conversions
      Plot original dtypes status, variable warning due high cardinality, save memory usage, final dtypes status.
    """
    # 0- Original dtypes
    # print(Fore.GREEN + "Input dtypes" + Style.RESET_ALL)
    # print(df.dtypes)
    # print("\n")
    print(Fore.RED + "Higha Cardinality, categorical features with levels > 15" + Style.RESET_ALL)

    # 1- Original memory_usage in MB
    start_mem = df.memory_usage().sum() / 1024 ** 2

    # 2- Numerical Types
    numerics = ["int8", "int16", "int32", "int64", "float16", "float32", "float64"]
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == "int": # First 3 characters
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if (c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max):
                    df[col] = df[col].astype(np.float32)
                #elif (c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max):
                #    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    # 3- Categorical Types
    high_card_vars = 0
    for col in df.select_dtypes(exclude = ["int8", "int16", "int32", "int64", "float16", "float32", "float64", "datetime64[ns]"]):
        categories = list(df[col].unique())
        cat_len = len(categories)
        if cat_len >= 2 and cat_len < 15:
           df[col] = df[col].astype("category")
        else:
          high_card_vars =+ 1
          # Print hight cardinality variables, amount of levels and a sample of 50 firts categories
          print(f"Look at: {Fore.RED + col + Style.RESET_ALL}, {cat_len} levels = {categories[:50]}")
    if high_card_vars == 0:
      print(Fore.GREEN + "None" + Style.RESET_ALL)
    else:
      pass

    # 4- Final memory_usage in MB
    end_mem = df.memory_usage().sum() / 1024 ** 2
    if verbose:
        print("\n")
        print(f"{Fore.RED}Initial memory usage: {start_mem:.2f} MB{Style.RESET_ALL}")
        print(f"{Fore.BLUE}Memory usage decreased to {end_mem:.2f} MB ({ 100 * (start_mem - end_mem) / start_mem:.1f}% reduction){Style.RESET_ALL}")
        #print("\n")
        #print(Fore.GREEN + "Output dtypes" + Style.RESET_ALL)
        #print(df.dtypes)
        print("\n")

    # 5. Feature types
    print(Fore.GREEN + "Variable types" + Style.RESET_ALL)
    numerical_vars = len(df.select_dtypes(include = ["number"]).columns)
    categorical_vars = len(df.select_dtypes(include = ["category", "object"]).columns)
    datetime_vars = len(df.select_dtypes(include = ["datetime64[ns]"]).columns)
    print(f"Numerical Features: {numerical_vars}")
    print(f"Categorical Features: {categorical_vars}")
    print(f"Datetime Features: {datetime_vars}")

    return df

### d.2 - Multicollinearity remove class

In [32]:
class RemoveMulticollinearity(BaseEstimator, TransformerMixin):
  """
  Summary:
  Class to create a sklearn custom transformer. Features with the VIF (variance inflation factor)
  higher than 10 are removed, it removes all except the feature with the highest correlation to y or target.
  Calculation based on training dataset.
  Note:
    This transformer always should be applied after categorical encoding, such as one-hot encoding.
  """
  def __init__(self, threshold: int = 5, method: str = 'spearmanr'):
    """
    Attributes:
        threshold (str, 10): Variance inflation factor threshold.
        method (str, default = spearmanr): Correlation method.
        feature_indices_ (np.array, default = None): Features indices to retain
    """
    self.threshold = threshold
    self.method = method
    self.feature_indices_ = None
    self.features_remove_ = None

  def fit(self, X: np.array = None, y: np.array = None):
    """
    Parameters:
      X_train (np.array, efault = None): Dataset of transformed predictors.
      y_train (np.array, default = None): Dataset of transformed target.

    Return:
      None.
    """
    # 1. Input features indeces; converting target to numpy array.
    self.feature_indices_ = np.arange(X.shape[1])
    y_train_array = y_train.to_numpy()

    # 2. VIF calculation
    vif = np.array([variance_inflation_factor(X, i) for i in range(X.shape[1])])

    # 3. Correlation between target and predictors
    corr_y = np.array([abs(spearmanr(X[:, i], y_train_array)[0].round(3)) for i in range(X.shape[1])])

    # 4. Colum stack: features, vif and correlation
    final = np.column_stack((self.feature_indices_, vif, corr_y)).round(3)

    # 5. Aplying filter VIF <= 10, exclude highest correlation to y or target
    mask = np.where((final[:, 1] > 10) & (final[:, 2] != final[:, 2].max()), final[:, 0], -1) > 0
    self.feature_indices_ = self.feature_indices_[~mask]
    return self

  def transform(self, X):
    """
    Return:
      numpy array with multicollinearity features removed.
    """
    return X[:, self.feature_indices_]

### d.3 - Metrics for evaluation model

In [33]:
# d.3 Metrics for evaluation model - binary calification
def binary_classification_metrics(y_true, y_pred, average: str = "binary", pos_label: str | int = None) -> pd.DataFrame:
  """
  Summary:
    Function to calculate multiple metrics to assess the performance of classification models on unseen data.
    Requiered import pandas and sklearn metrics module.
  Parameters:
    y_true (array, pd.DataFrame): original target dataset.
    y_pred (array, pd.DataFrame): predicted target dataset.
    average (str, default = binary): type of averaging performed on the data, important for multiclass classification.
    pos_label (str|int, default = None): positive class in binary classification.
  Returns:
    DataFrame contains metrics of classificatioin performance.
  """
  # Calculate specificity and Negative Predictive Value for binary classification problems
  tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()

  specificity = tn / (tn + fp)
  npv = tn / (tn + fp) # Negative Predictive Value

  # Metrics to evaluate:
  metrics_labels = ["Accuracy", "Precision", "Recall", "Specificity", "Negative Predictive Value", "F1-Score", "Balance Accuracy", "Jaccard-Score", "Matthews Corr Coef", "Cohen’s kappa"]

  # Average parameter
  average = average # "binary" for binary classfication, "micro, macro, ..." form multiclass classification

  # Calculations
  metrics_values = [accuracy_score(y_true, y_pred),
                    precision_score(y_true, y_pred, average = average, pos_label = pos_label),
                    recall_score(y_true, y_pred, average = average, pos_label = pos_label),
                    specificity,
                    npv,
                    f1_score(y_true, y_pred, average = average, pos_label = pos_label),
                    balanced_accuracy_score(y_true, y_pred),
                    jaccard_score(y_true, y_pred, average = average, pos_label = pos_label),
                    matthews_corrcoef(y_true, y_pred),
                    cohen_kappa_score(y1 = y_true, y2 = y_pred)]

  metrics_df = pd.DataFrame(data = {"Metric": metrics_labels,
                                    "Value": metrics_values})

  return metrics_df.round(3)

## 1 - Load data

In [ ]:
# 1.1 - Import manin file from Google drive

# Mount Google Drive
drive.mount('/content/drive')


# Specify the source path in Google Drive
drive_filepath = '/content/drive/MyDrive/DataScience_Portfolio/04-Churn_Telco_Analysis/'

# Specify the destination path in Colab
colab_filepath = '/content/'

# Copy the file from Google Drive to Colab
try:
  shutil.copy(src = drive_filepath + '/churn_data.txt', dst = colab_filepath + '/churn_data.txt')
except:
  pass

In [45]:
%%time
# 1.2 - Load data
churn_df = pl.read_csv(source = 'churn_data.txt',
                       has_header = True,
                       separator = "|",
                       try_parse_dates = True,
                       encoding = "utf8") \
             .drop(columns = ["SUBSCRIBER_ID"]) \
             .drop_nulls(subset = ['churn']) \
             .with_columns(pl.col(['region', 'churn', 'canal', 'bandas', 'tipo_gross_adds']).cast(pl.Categorical)) \
             .to_pandas() \
             .pipe(dtype_conversion)

Higha Cardinality, categorical features with levels > 15
Look at: region, 34 levels = ['Bogota D.C.', 'Santander', 'Antioquia', nan, 'Cundinamarca', 'Quindio', 'Valle Del Cauca', 'Arauca', 'Bolivar', 'Atlantico', 'Tolima', 'Huila', 'Meta', 'Putumayo', 'Boyaca', 'Caldas', 'Cordoba', 'Nariño', 'Magdalena', 'Risaralda', 'Cauca', 'La Guajira', 'Norte De Santander', 'Caqueta', 'Cesar', 'Guaviare', 'Sucre', 'Amazonas', 'Guainia', 'Vichada', 'Choco', 'Casanare', 'Providencia Islas', 'Vaupes']


Initial memory usage: 175.54 MB
Memory usage decreased to 87.48 MB (50.2% reduction)


Variable types
Numerical Features: 38
Categorical Features: 5
Datetime Features: 0
CPU times: user 1.95 s, sys: 1.02 s, total: 2.97 s
Wall time: 2.55 s


In [ ]:
churn_df.head()

,cant_sem_datos,prom_gb_tt,prom_gb_ran,prom_%_propia,continuidad_traf,variacion_datos_8s,cons_ult_sem,contrafico,mean_minutos_voz_in,mean_llamadas_in,...,bandas,tipo_gross_adds,semanas_antiguedad,semanas_contactos_pqr,contactos_ult_semana_pqr,seg_llamadas_ofertas_ult_semana,var_seg_llamadas_ofertas_4_semanas,porc_descuento_activo,jineteo,churn
0,1.0,0.603,0.000,1.000,1.000,0.000000,0.603,1,29.511000,5.571,...,Fdd Band 28,Gross Adds Migracion,1.0,1.0,2.0,0.0,0.0,0.0,1.0,No
1,5.0,3.369,0.075,0.975,1.250,-17.405001,2.783,1,52.870998,18.875,...,Ninguna,Gross Adds Nueva,4.0,0.0,0.0,0.0,0.0,0.0,1.0,No
2,8.0,0.832,0.019,0.983,0.500,-48.924999,0.425,1,64.025002,18.875,...,Ninguna,Gross Adds Migracion,16.0,0.0,0.0,0.0,0.0,0.0,1.0,No
3,8.0,6.824,0.328,0.933,0.727,-60.625000,2.687,1,148.910004,37.000,...,Fdd Band 28,Gross Adds Portacion,11.0,0.0,0.0,0.0,0.0,0.0,1.0,No
4,8.0,0.917,0.192,0.810,0.444,65.065002,1.514,1,77.785004,15.500,...,Fdd Band 28,Gross Adds Portacion,18.0,0.0,0.0,0.0,0.0,0.0,1.0,No


## 2 - Churn prediction

### 2.1 - MLflow UI Setup

In [34]:
# 1. Set the tracking URI to the default local file system
mlflow.set_tracking_uri('file:/content/mlruns')

In [ ]:
# 2. Set an experiment name and autolog
mlflow.set_experiment("04-Churn_Telco_Analysis")
mlflow.autolog(log_input_examples = True, log_model_signatures = True)

### 2.2 - Variables types

In [46]:
# Variables types

# Target feature
target_var = ["churn"]

# Datetime features
datetime_vars = churn_df.select_dtypes(include = "datetime64[ns]").columns.tolist()

# Numerical features
numerical_vars = churn_df.select_dtypes(include = "number").columns.tolist()

# Categorical features
nominal_vars = churn_df.select_dtypes(include = "category").columns.tolist()
nominal_vars.pop(-1) # Exclude target variable, instantiated above

# 2.2 Reorder DataFrame
new_col_order = target_var + numerical_vars + nominal_vars
churn_df = churn_df.reindex(columns = new_col_order)

In [47]:
len(churn_df.columns)

43

### 2.3 - Train - Test split

In [48]:
# Target and independent variables
X = churn_df.copy(deep = True).drop(labels = ["churn"], axis = 1)
y = churn_df.copy(deep = True).loc[:, "churn"]

In [49]:
# Train test split
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    train_size = 0.75,
                                                    test_size = 0.25,
                                                    stratify = y,
                                                    random_state = 123456)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(446747, 42) (148916, 42) (446747,) (148916,)


### 2.4 - Pipeline Cache

In [53]:
# Create a temporary folder to store the transformers of the pipeline
memory = Memory(location = "cachedir", verbose = 1)

In [51]:
# Remove the cache directory at the end of your script
# Don't run
rmtree("cachedir")

#### Import pipeline cache files

In [ ]:
# Mount Google Drive
drive.mount('/content/drive')

# Specify the source path in Google Drive
drive_folderpath_cache = '/content/drive/MyDrive/DataScience_Portfolio/04-Churn_Telco_Analysis/'

# Specify the destination path in Colab
colab_folderpath_cache = '/content/'

# Copy the folder from Google Drive to Colab
try:
  shutil.copytree(drive_folderpath_cache + '/cachedir', colab_folderpath_cache + '/cachedir')
except:
  pass

#### Export pipeline cache files

In [83]:
# Mount Google Drive
drive.mount('/content/drive')

# Specify the temporary file or folder path in Colab
temp_folderpath_cache = '/content/cachedir'

# Specify the destination path in Google Drive
drive_folderpath_cache = '/content/drive/MyDrive/DataScience_Portfolio/04-Churn_Telco_Analysis/cachedir'

# Remove the existing folder in Google Drive
try:
  shutil.rmtree(drive_folderpath_cache)
except:
  pass

# Copy the temporary folder to Google Drive
shutil.copytree(temp_folderpath_cache, drive_folderpath_cache)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


'/content/drive/MyDrive/DataScience_Portfolio/04-Churn_Telco_Analysis/cachedir'

### 2.5 - Encoding target variable

In [55]:
# 1. Instantiate Encoder
target_encoder = LabelEncoder()

# 2. y train encoder
y_train_encoded = target_encoder.fit_transform(y_train)
print(target_encoder.classes_)
y_train = pd.DataFrame(data = y_train_encoded, columns = ["churn"])
print(y_train.head())

# 3.3 y test encoder
y_test_encoded = target_encoder.transform(y_test)
y_test = pd.DataFrame(data = y_test_encoded, columns = ['churn'])
print(y_test.head())

['No' 'Sí']
   churn
0      0
1      0
2      0
3      0
4      0
   churn
0      0
1      0
2      0
3      0
4      0


### 2.6 - Numerical Pipeline

In [ ]:
# In this case, don't required

### 2.7 - Ordinal Pipeline

In [ ]:
# In this case, don't required

### 2.7 - Nominal Pipeline

In [57]:
# Nominal pipeline: one-hot-encoding

nominal_pipeline = make_pipeline(
    # 1. Nominal encoder
    OneHotEncoder(categories = "auto",
                  drop = "first", # "first" or "None", Specifies a methodology to use to drop one of the categories per feature
                  sparse_output = False,
                  dtype = "float", # Desired dtype of output
                  handle_unknown = 'infrequent_if_exist',
                  min_frequency = 0.03,
                  max_categories = 30),
)

nominal_pipeline

Pipeline(steps=[('onehotencoder',
                 OneHotEncoder(drop='first', dtype='float',
                               handle_unknown='infrequent_if_exist',
                               max_categories=30, min_frequency=0.03,
                               sparse_output=False))])

### 2.8 - Transformers

In [58]:
# 1 Combine ordinal and nominal pipelines
transformations = ColumnTransformer(
    transformers=[("numerical", "passthrough", numerical_vars), # No transformation required for numerical variables
                  # ("ordinal", ordinal_pipeline, ordinal_vars), # No ordinal data
                  ("nominal", nominal_pipeline, nominal_vars)],
    #remainder = 'passthrough'
)

### 2.9 - Full pipeline

In [ ]:
# Calculate the class imbalance ratio based on  XGBoostClassifier
# https://xgboost.readthedocs.io/en/stable/tutorials/param_tuning.html
positive_samples = sum(y_train.loc[:, 'churn'] == 1)
negative_samples = sum(y_train.loc[:, 'churn'] == 0)
imbalance_ratio = negative_samples / positive_samples

In [189]:
# Full pipeline

full_pipeline = make_pipeline(
    # 1. Numerical and categorical (nominal + ordinal) transformers
    transformations,

    # 2. Impute missing values
    IterativeImputer(estimator = BayesianRidge(),
                     max_iter = 10,
                     random_state = 123456),

    # 3. Feature selector that removes all low-variance features
    VarianceThreshold(threshold = 0.1),

    # 4. Remove Multicollinearity: doesn't work in this case
    # RemoveMulticollinearity(threshold = 10),

    # 5. Scale features
    RobustScaler(with_centering = True,
                 with_scaling = True,
                 quantile_range = (25.0, 75.0),
                 copy = True,
                 unit_variance = False),

    # 6. Transform features
    PowerTransformer(method = 'yeo-johnson',
                     standardize = True,
                     copy = True),

    # 7. Dimensionality reduction
    PCA(n_components = 0.95),
    # UMAP(n_components = 5),

    # 8. Feature selection: univariate feature selection ~ 50% of features
    SelectKBest(f_classif, k = 20),

    # 9. Classifier
    LogisticRegression(class_weight ='balanced',
                       n_jobs = -1),

    # 10. Memory cache
    memory = memory
)

full_pipeline

Pipeline(memory=Memory(location=cachedir/joblib),
         steps=[('columntransformer',
                 ColumnTransformer(transformers=[('numerical', 'passthrough',
                                                  ['cant_sem_datos',
                                                   'prom_gb_tt', 'prom_gb_ran',
                                                   'prom_%_propia',
                                                   'continuidad_traf',
                                                   'variacion_datos_8s',
                                                   'cons_ult_sem', 'contrafico',
                                                   'mean_minutos_voz_in',
                                                   'mean_llamadas_in',
                                                   'mean_llamadas_in_num_dif',
                                                   'mean_porcent_lla...
                ('iterativeimputer',
                 IterativeImputer(estimator=BayesianRidge(),
                                  random_state=123456)),
                ('variancethreshold', VarianceThreshold(threshold=0.1)),
                ('robustscaler', RobustScaler()),
                ('powertransformer', PowerTransformer()),
                ('pca', PCA(n_components=0.95)),
                ('selectkbest', SelectKBest(k=20)),
                ('logisticregression',
                 LogisticRegression(class_weight='balanced', n_jobs=-1))])

#### XGBoost Classifier

In [ ]:
# 2. Estimator hyperparameters
full_pipeline.named_steps['xgbclassifier'].get_params()

In [ ]:
xgb.XGBClassifier(booster = "gbtree",
                  objective = "binary:logistic",
                  scale_pos_weight = imbalance_ratio,
                  early_stopping_rounds = 70)

In [132]:
# Grid parameter if BayesSearchCV is used
xgb_param_bayes_space = {"selectkbest__k": Integer(low=10, high=43),
                         'pca__n_components': Real(low=0.4, high=0.95, prior="uniform"),
                         "xgbclassifier__max_depth": Integer(low=4, high=10),
                         "xgbclassifier__min_child_weight": Integer(low=0, high=10),
                         "xgbclassifier__subsample": Real(low=0.4, high=1.0, prior="uniform"),
                         "xgbclassifier__colsample_bytree": Real(low=0.4, high=1.0, prior="uniform"),
                         "xgbclassifier__n_estimators": Integer(low=100, high=1000),
                         "xgbclassifier__learning_rate": Real(low=0.01, high=0.3, prior="uniform"),
                         "xgbclassifier__gamma": Integer(low=0, high=1),
                         "xgbclassifier__reg_lambda": Integer(low=1, high=5),
                         "xgbclassifier__reg_alpha": Integer(low=0, high=5)}

#### LightGBM Classifier

In [134]:
# 2. Estimator hyperparameters
full_pipeline.named_steps['lgbmclassifier'].get_params()

{'boosting_type': 'gbdt',
 'class_weight': None,
 'colsample_bytree': 1.0,
 'importance_type': 'split',
 'learning_rate': 0.1,
 'max_depth': -1,
 'min_child_samples': 20,
 'min_child_weight': 0.001,
 'min_split_gain': 0.0,
 'n_estimators': 100,
 'n_jobs': -1,
 'num_leaves': 31,
 'objective': None,
 'random_state': None,
 'reg_alpha': 0.0,
 'reg_lambda': 0.0,
 'silent': 'warn',
 'subsample': 1.0,
 'subsample_for_bin': 200000,
 'subsample_freq': 0,
 'scale_pos_weight': 29.175413711583925}

In [141]:
lgb.LGBMClassifier(scale_pos_weight = imbalance_ratio, # Like in XGBoostClassifier
                   early_stopping_round = 70),

(LGBMClassifier(early_stopping_round=70, scale_pos_weight=29.175413711583925),)

In [135]:
# Grid parameter if BayesSearchCV is used
lgb_param_bayes_space = {"selectkbest__k": Integer(low=10, high=43),
                         'pca__n_components': Real(low=0.4, high=0.95, prior="uniform"),
                         "lgbmclassifier__colsample_bytree": Real(low=0.4, high=1.0, prior="uniform"),
                         "lgbmclassifier__learning_rate": Real(low=0.01, high=0.3, prior="uniform"),
                         "lgbmclassifier__max_depth": Integer(low=4, high=12),
                         "lgbmclassifier__num_leaves": Integer(low=10, high=200),
                         "lgbmclassifier__min_child_samples":Integer(low=20, high=100),
                         "lgbmclassifier__min_child_weight": Real(low=0.0, high=0.3, prior="uniform"),
                         "lgbmclassifier__min_split_gain": Real(low=0.0, high=0.1, prior="uniform"),
                         "lgbmclassifier__n_estimators": Integer(low=100, high=1000),
                         "lgbmclassifier__reg_alpha": Real(low=0.0, high=1, prior="uniform"),
                         "lgbmclassifier__reg_lambda": Real(low=0.0, high=1, prior="uniform"),
                         "lgbmclassifier__subsample": Real(low=0.3, high=1.0, prior="uniform")}

#### CatBoost Classifier

In [159]:
full_pipeline.named_steps['catboostclassifier'].get_params()

{'class_weights': [1, 29.175413711583925]}

In [160]:
catb.CatBoostClassifier(class_weights = [1, imbalance_ratio])

In [158]:
# Grid parameter if BayesSearchCV is used
cat_param_bayes_space = {"selectkbest__k": Integer(low=10, high=43),
                         'pca__n_components': Real(low=0.4, high=0.95, prior="uniform"),
                         "catboostclassifier__learning_rate": Real(low=0.01, high=0.3, prior="uniform"),
                         "catboostclassifier__depth": Integer(low=4, high=12),
                         "catboostclassifier__l2_leaf_reg": Integer(low=1, high=10),
                         "catboostclassifier__iterations": Integer(low=100, high=1000),
                         "catboostclassifier__random_strength": Real(low=0.1, high=1, prior="uniform"),
                         "catboostclassifier__bagging_temperature": Integer(low=1, high=6),
                         "catboostclassifier__leaf_estimation_method": Categorical(["Newton", None, 'Gradient', 'Exact']),
                         "catboostclassifier__min_data_in_leaf": Integer(low=20, high=50)}

#### AdaBoost Classifier

In [177]:
full_pipeline.named_steps['adaboostclassifier'].get_params()

{'algorithm': 'SAMME.R',
 'base_estimator': 'deprecated',
 'estimator': None,
 'learning_rate': 1.0,
 'n_estimators': 50,
 'random_state': None}

In [179]:
AdaBoostClassifier(estimator = 'GaussianNB')

AdaBoostClassifier(estimator='GaussianNB')

In [ ]:
ada_param_bayes_space = {"selectkbest__k": Integer(low=10, high=43),
                         'pca__n_components': Real(low=0.4, high=0.95, prior="uniform"),
                         "adaboostclassifier__learning_rate": Real(low=0.01, high=1, prior="uniform"),
                         "adaboostclassifier__n_estimators": Integer(low=50, high=200)}

#### RandomForestClassifier

In [149]:
full_pipeline.named_steps['randomforestclassifier'].get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': 'balanced',
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'sqrt',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [147]:
RandomForestClassifier(class_weight = 'balanced',
                       n_jobs = -1)

RandomForestClassifier(class_weight='balanced')

In [ ]:
rf_param_bayes_space = {"selectkbest__k": Integer(low=10, high=43),
                        'pca__n_components': Real(low=0.4, high=0.95, prior="uniform"),
                        "randomforestclassifier__n_estimators": Integer(low=100, high=1000),
                        "randomforestclassifier__max_depth": Integer(low=4, high=20),
                        "randomforestclassifier__min_samples_split": Integer(low=2, high=20),
                        "randomforestclassifier__min_samples_leaf": Integer(low=1, high=10),
                        "randomforestclassifier__class_weight": Categorical(["balanced", None]),
                        "randomforestclassifier__min_impurity_decrease": Real(low=0, high=0.5),
                        "randomforestclassifier__ccp_alpha": Real(low=0, high=0.15)}

#### Logistic Regression

In [171]:
full_pipeline.named_steps['logisticregression'].get_params()

{'C': 1.0,
 'class_weight': 'balanced',
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'l1_ratio': None,
 'max_iter': 100,
 'multi_class': 'auto',
 'n_jobs': -1,
 'penalty': 'l2',
 'random_state': None,
 'solver': 'lbfgs',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

In [167]:
LogisticRegression(class_weight ='balanced',
                   n_jobs = -1)

LogisticRegression(class_weight='balanced', n_jobs=-1)

In [190]:
lr_param_bayes_space = {"selectkbest__k": Integer(low=10, high=43),
                        'pca__n_components': Real(low=0.4, high=0.95, prior="uniform"),
                        "logisticregression__C": Integer(low=1, high=100),
                        "logisticregression__penalty": Categorical(["l1", "l2"]),
                        "logisticregression__tol":  Real(low=0.0001, high=0.02),
                        "logisticregression__max_iter": Integer(low=100, high=1000),
                        "logisticregression__solver": Categorical(['lbfgs', 'liblinear', 'newton-cg',' newton-cholesky', 'sag']),
                        "logisticregression__l1_ratio": Real(low=0, high=1)}

### 3.0 - Hyperparameter Tuning

In [60]:
# 0. Cross-validation: evaluating estimator performance
"""
https://scikit-learn.org/stable/modules/cross_validation.html#cross-validation-iterators
"""
# Cross-validation iterators for Independent and Identically Distributed (i.i.d.) data
# I will used alternative of cv iterators

# 5 k-fold cross validation
kf = KFold(n_splits = 5)

# Stratified folds: each set contains approximately the same percentage of samples of each target class as the complete set
skf = StratifiedKFold(n_splits = 5)

# Shuffle & Split: iterator will generate a user defined number of independent train / test dataset splits
ss = ShuffleSplit(n_splits = 5, test_size = 0.20, random_state = 123456)

# Stratified ShuffleSplit cross-validator
sss = StratifiedShuffleSplit(n_splits = 5, test_size= 0.2, random_state = 123456)

#### BayesSearchCV

In [191]:
# Bayesian optimization over hyperparameters
BayesSearchCV_pipeline = BayesSearchCV(estimator = full_pipeline,
                                       search_spaces = lr_param_bayes_space,
                                       n_iter = 150,
                                       scoring = "recall",
                                       cv = sss,
                                       refit = True,
                                       random_state = 123456,
                                       return_train_score = True)

### 3.1 - Experiments

In this section, using MLflow, I will to record and query experiments: code, data, config, and results.

In [186]:
# 1 - Fit the model.
#Fit all the transformers one after the other and transform the data. Finally, fit the transformed data using the final estimator.

%%time
description = 'LogisticRegression with tuned hyperparameters, all pipeline steps and class_weight implemented'
with mlflow.start_run(run_name = "13-LogisticRegression_Tuned", description = description) as run:
  full_pipeline.fit(X_train, y_train)
  y_pred_raw = full_pipeline.predict(X_test)

  # Evaluation metrics
  y_prob_pred = full_pipeline.predict_proba(X_test)[:, 1]
  test_auc = roc_auc_score(y_true = y_test, y_score = y_prob_pred)
  tn, fp, fn, tp = confusion_matrix(y_test, y_pred_raw).ravel()
  test_specificity = tn / (tn + fp)
  test_npv = tn / (tn + fp) # Negative Predictive Value
  test_accuracy_score = accuracy_score(y_test, y_pred_raw)
  test_precision_score = precision_score(y_test, y_pred_raw)
  test_recall_score = recall_score(y_test, y_pred_raw)
  test_f1_score = f1_score(y_test, y_pred_raw)
  test_balanced_accuracy_score = balanced_accuracy_score(y_test, y_pred_raw)
  test_jaccard_score = jaccard_score(y_test, y_pred_raw)
  test_matthews_corrcoef = matthews_corrcoef(y_test, y_pred_raw)
  test_cohen_kappa_score = cohen_kappa_score(y1 = y_test, y2 = y_pred_raw)

  mlflow.log_metric('auc_X_test', test_auc)
  mlflow.log_metric('specificity_X_test', test_specificity)
  mlflow.log_metric('npv_X_test', test_npv)

2023/07/28 08:08:43 WARNING mlflow.utils: Truncated the value of the key `steps`. Truncated value: `[('columntransformer', ColumnTransformer(transformers=[('numerical', 'passthrough',
                                 ['cant_sem_datos', 'prom_gb_tt', 'prom_gb_ran',
                                  'prom_%_propia', 'continuidad_traf',
                                  'variacion_datos_8s', 'cons_ult_sem',
                                  'contrafico', 'mean_minutos_voz_in',
                                  'mean_llamadas_in',
                                  'mean_llamadas_in_num_dif',
  ...`
2023/07/28 08:08:43 WARNING mlflow.utils: Truncated the value of the key `columntransformer`. Truncated value: `ColumnTransformer(transformers=[('numerical', 'passthrough',
                                 ['cant_sem_datos', 'prom_gb_tt', 'prom_gb_ran',
                                  'prom_%_propia', 'continuidad_traf',
                                  'variacion_datos_8s', 'cons_ult_sem',
 

CPU times: user 47.1 s, sys: 10.7 s, total: 57.7 s
Wall time: 58.5 s


In [187]:
# 2 - Open a secure tunnels from public MLflow URL to localhost:
ngrok.kill()
ngrok_tunnel = ngrok.connect(addr="5000", proto= "http", bind_tls = True)
ngrok.set_auth_token("2Siae72MMz4I2KtDOd9oxxV52gU_4UHBVTbE7bFnfYw4PDvuv")
print("MLflow Tracking UI: ", ngrok_tunnel.public_url)

# 3. View MLflow runs and experiments
!mlflow ui

MLflow Tracking UI:  https://c66c-35-202-235-201.ngrok-free.app
[2023-07-28 08:09:55 +0000] [113136] [INFO] Starting gunicorn 20.1.0
[2023-07-28 08:09:55 +0000] [113136] [INFO] Listening at: http://127.0.0.1:5000 (113136)
[2023-07-28 08:09:55 +0000] [113136] [INFO] Using worker: sync
[2023-07-28 08:09:55 +0000] [113140] [INFO] Booting worker with pid: 113140
[2023-07-28 08:09:55 +0000] [113142] [INFO] Booting worker with pid: 113142
[2023-07-28 08:09:55 +0000] [113143] [INFO] Booting worker with pid: 113143
[2023-07-28 08:09:55 +0000] [113144] [INFO] Booting worker with pid: 113144

[2023-07-28 08:15:08 +0000] [113136] [INFO] Handling signal: int
Aborted!
[2023-07-28 08:15:08 +0000] [113142] [INFO] Worker exiting (pid: 113142)
[2023-07-28 08:15:08 +0000] [113140] [INFO] Worker exiting (pid: 113140)
[2023-07-28 08:15:08 +0000] [113144] [INFO] Worker exiting (pid: 113144)
[2023-07-28 08:15:08 +0000] [113143] [INFO] Worker exiting (pid: 113143)
[2023-07-28 08:15:09 +0000] [113136] [INFO] 

#### Import mlruns from Google Drive

In [ ]:
# Mount Google Drive
drive.mount('/content/drive')

# Specify the source path in Google Drive
drive_folderpath_mlruns = '/content/drive/MyDrive/DataScience_Portfolio/04-Churn_Telco_Analysis/'

# Specify the destination path in Colab
colab_folder_path = '/content/'

# Copy the folder from Google Drive to Colab
try:
  shutil.copytree(drive_folderpath_mlruns + '/mlruns', colab_folder_path + '/mlruns')
except:
  pass

#### Export mlruns to Google Drive

In [ ]:
# Specify the temporary file or folder path in Colab
temp_folderpath_mlruns = '/content/mlruns'

# Specify the destination path in Google Drive
drive_folderpath_mlruns = '/content/drive/MyDrive/DataScience_Portfolio/04-Churn_Telco_Analysis/mlruns'

# Remove the existing folder in Google Drive
try:
  shutil.rmtree(drive_folderpath_mlruns)
except:
  pass

# Copy the temporary folder to Google Drive
shutil.copytree(temp_folderpath_mlruns, drive_folderpath_mlruns)